# Mini Batches
Iterate over a data stream in mini batches

In [1]:
import pandas as pd
from river import datasets
from river_torch import regression
from torch import nn
from river import compose
from river import preprocessing
from itertools import islice

In [2]:
class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features,5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 1)
        self.relu = nn.ReLU()

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.relu(X)
        return X

def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [3]:
dataset = datasets.Bikes()

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= regression.Regressor(module=MyModule, loss_fn="mse", optimizer_fn="sgd")
model_pipeline

Pipeline (
  Select (
    clouds
    humidity
    pressure
    temperature
    wind
  ),
  StandardScaler (
    with_std=True
  ),
  Regressor (
    module=<class '__main__.MyModule'>
    loss_fn="mse_loss"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.001
    device="cpu"
    seed=42
  )
)

In [4]:
for batch in batcher(dataset.take(5000),5):
    x,y = zip(*batch)
    x = pd.DataFrame(x)
    y_pred = model_pipeline.predict_many(X=x)
    model_pipeline.learn_many(X=x, y=y)

C:\Users\kulbach\Documents\environments\river-torch\lib\site-packages\river\preprocessing\scale.py:243: RuntimeWarning: invalid value encountered in double_scalars
  stds = np.array([self.vars[c] ** 0.5 for c in X.columns])
C:\Users\kulbach\Documents\projects\IncrementalLearning\river-torch\river_torch\regression\regressor.py:141: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss_fn(y_pred, y)
